# 머신러닝 with Python 
## 09 앙상블 학습 스태킹 연습 - wine 데이터

##### 스태킹 알고리즘을 활용하여 wine을 분류하는 모형을 만들어보고자 한다.

(책 p.295~ 원래 실습은 유방암 양성음성 관련 데이터였으나 모형들과 관련한 의문이 남아서 다른 데이터로도 실행을 해보고자 한다.) 

---

하단부 실험에서는 이전 스태킹 실험들과 마찬가지로, 학습기 종류 자체나 학습의 순서 등을 변경하는 실험을 진행하였다.

> 배운점: garbage in, garbage out과 마찬가지로 베이스 학습기가 분류를 잘 했다면 좋은 output을 받은 메타 학습기가 학습을 잘 할 것이다. 


### 데이터 불러오기

In [1]:
from sklearn import datasets
raw_wine = datasets.load_wine()

In [2]:
# 데이터 셋 내 피처 살펴보기
raw_wine.feature_names

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

### 피처, 타깃 데이터 지정

In [3]:
X = raw_wine.data
y = raw_wine.target

### 트레이닝/테스트 데이터 분할

### 데이터 표준화

In [4]:
from sklearn.model_selection import train_test_split              # 데이터 분할을 위한 함수
from sklearn.preprocessing import StandardScaler                  # 데이터 표준화를 위한 함수

X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0) # 데이터 분할

std_scale = StandardScaler()                                      # 표준화 스케일러 선언
std_scale.fit(X_tn)                                               # 트레이닝 피처(데이터)를 기준으로 표준화 스케일러 적합

X_tn_std = std_scale.transform(X_tn)                              # 트레인, 테스트 데이터를 표준화 스케일러 처리
X_te_std = std_scale.transform(X_te)

### 데이터 학습 (베이스러너: SVM, Gnb | 메타러너: LogisticRegression)

In [5]:
from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('svm', base_clf1),('gnb', base_clf2)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) # 분류 문제를 해결해야 할 경우 StackingRegressor 사용

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

StackingClassifier(estimators=[('svm', SVC(kernel='linear', random_state=0)),
                               ('gnb', GaussianNB())],
                   final_estimator=LogisticRegression(random_state=0))

### 데이터 예측

In [6]:
pred_stacking = clf_stacking.predict(X_te_std)
print(pred_stacking)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [7]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_stacking)
print(accuracy)

0.9555555555555556


### confusion matrix 확인

In [8]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_stacking)
print(conf_matrix)

[[16  0  0]
 [ 1 19  1]
 [ 0  0  8]]


### 분류 리포트 확인

In [9]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_stacking)
print(class_report)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45



### 결과: 0.9555

오 역시 그렇고 그런 성능이 나왔습니다. (90%대의 성능에 너무 익숙해진건 아닐까...?)

비슷한 데이터 분포로 여러 데이터들을 만들어서 실험해 볼 수 있으면 좋을텐데.

이어서 다른 모형들의 조합으로도 해본다.

# 실험- 학습기 개수 변경
## 실험1 : 베이스 학습기를 SVC만 사용했을 때

In [10]:
from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
# from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
# base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('svm', base_clf1)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("베이스 학습기- SVM만 사용했을 경우 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
# print("========================================================")

베이스 학습기- SVM만 사용했을 경우 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



*?????????? 머선일이고?*

## 실험2: 베이스 학습기를 GaussianNB만 사용했을 때

In [11]:
# from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


# base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('gnb', base_clf2)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("베이스 학습기- 가우시안 나이브 베이즈만 사용했을 경우 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
# print("========================================================")

베이스 학습기- 가우시안 나이브 베이즈만 사용했을 경우 정확도:  0.9333333333333333
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.89      1.00      0.94        16
           1       1.00      0.86      0.92        21
           2       0.89      1.00      0.94         8

    accuracy                           0.93        45
   macro avg       0.93      0.95      0.94        45
weighted avg       0.94      0.93      0.93        45



### 위의 실험 결과

유방암 분류하는 것과 비슷하게, 서포트 벡터 머신을 사용했을 때 보다 나이브 베이즈 모형을 베이스 학습기로 사용했을 때 성능이 더 좋지 않게 나왔다.

문제는 SVM을 베이스 학습기로 사용했을 때의 성능이 1.0이 나와버렸다는 것인데, 이 분류를 위해서 가장 잘 맞는 모형인 것일까?

좋은 데이터가 메타 학습기에 들어갔기 때문에 정확도가 높게 나왔겠지?

---

# 실험3: 학습기 순서 변경

In [12]:
from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('gnb', base_clf2), ('svc', base_clf1)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("베이스 학습기 순서 변경 후 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
# print("========================================================")

베이스 학습기 순서 변경 후 정확도:  0.9555555555555556
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45



### 결과

마찬가지로 순서가 상관이 없음이 밝혀진 것 같다. 베이스 러너- 메타 러너의 순서로 가긴 하지만, 베이스 러너들 간에는 순서 없이 말그대로 결과를 종합하는 것이기 때문인 것 같다.

# 실험- 기본 학습기와 메타 학습기 조합 변경

베이스 학습기들의 결과가 나온 것을 메타 학습기가 받는다고 했을 때, 

같은 종류의 학습기가 순차적으로 데이터를 받아 분류한다면 결과가 어떨지에 대한 궁금증과,

세 가지 분류 알고리즘들의 조합에 따라 이 데이터 셋에서는 어떤 성능차이가 있을지 궁금해졌다.

## 실험4: 셋 중 메타 학습기를 변경했을 때

In [13]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형

clf_svc = svm.SVC(kernel = 'linear', random_state = 0)
clf_gnb = GaussianNB()
clf_lr = LogisticRegression(random_state = 0)


# 메타 학습기가 SVC 일 때
clf_stacking = StackingClassifier(estimators = [('gnb', clf_gnb), ('lr', clf_lr)], 
                                  final_estimator = svm.SVC(kernel = 'linear', random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기: ", "SVM")
print("베이스 학습기: ", "Gaussian Naive Bayes, Logistic Regression")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
print("========================================================")


# 메타 학습기가 Naive Bayes 일 때
clf_stacking = StackingClassifier(estimators = [('svc', clf_svc), ('lr', clf_lr)], final_estimator = GaussianNB()) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기: ", "Gaussian Naive Bayes")
print("베이스 학습기: ", "SVM, Logistic Regression")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)

메타 학습기:  SVM
베이스 학습기:  Gaussian Naive Bayes, Logistic Regression
모델 정확도:  0.9555555555555556
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45

메타 학습기:  Gaussian Naive Bayes
베이스 학습기:  SVM, Logistic Regression
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



### 결과

신기한 결과가 나왔다. 메타 러너가 서포트 벡터 머신이라 하더라도 가우시안 나이브 베이즈가 베이스 학습기로 들어있을 때의 성능은 그저 그랬다. 

처음의 기본 설계 모델과 같은 성능과 같은 성능이다. 처음 모델의 성능을 보자면,
```
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45
```

실제로 정확도나 재현율도 클래스 별로 같게 나왔다. 베이스 러너간의 우위가 있지 않음에도, 성능의 차이에 영향을 이만큼 줄 만큼의 차이가 있음을 볼 수 있었다.

그렇다면 정말 같은 아이들끼리 붙여봤을떄, 어떨까?

## 실험5: 베이스 학습기와 메타 학습기가 같은 모형일 때

In [14]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형

clf_svc = svm.SVC(kernel = 'linear', random_state = 0) 
clf_gnb = GaussianNB()
clf_lr = LogisticRegression(random_state = 0)


# LogisticRegression
clf_stacking = StackingClassifier(estimators = [('lr', clf_lr)], 
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기 = 베이스 학습기: ", "Logistic Regression")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
print("========================================================")


# SVM
clf_stacking = StackingClassifier(estimators = [('svc', clf_svc)],
                                  final_estimator = svm.SVC(kernel = 'linear', random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기 = 베이스 학습기: ", "SVM")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
print("========================================================")


# Gaussian Naive Bayes
clf_stacking = StackingClassifier(estimators = [('gnb', clf_gnb)], 
                                  final_estimator = GaussianNB()) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기 = 베이스 학습기: ", "Gaussian Naive Bayes")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)

메타 학습기 = 베이스 학습기:  Logistic Regression
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

메타 학습기 = 베이스 학습기:  SVM
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

메타 학습기 = 베이스 학습기:  Gaussian Naive Bayes
모델 정확도:  0.9555555555555556
모델 분류 리포트
              precision    recall  f1-score   suppor

### 결과

다양한 피처들을 데리고 분류 문제를 품에 있어서 Gaussian Naive Bayes보다는 LogisticRegression과 SupportVectorMachine이 더 나은지도 모른다는 생각을 들게하는 실험 결과였다.

처음 내가 접했던, iris 데이터를 가지고도 해보아야겠다.